# Robust Decision Making

To solve decision problems using bounded probability, we first need some code to calculate lower and upper expectations themselves.

## Credal Sets

Here, for simplicity, we will always specify a lower expectation by means of a finite set of probability mass functions.
The lower expectation is then the minimal expectation with respect to this set.

Let's move to lower expectations. A credal set will then be a set of probability mass functions. For simplicity, we will also use a sequence to represent these in the code. For example, when coding, we will represent the credal set $\{(0.2, 0.2, 0.6), (0.4, 0.1, 0.5)\}$ as follows:

In [1]:
[[0.2, 0.2, 0.6], [0.4, 0.1, 0.5]]

[[0.2, 0.2, 0.6], [0.4, 0.1, 0.5]]

## Lower and Upper Expectations

The lower and upper expectation are simply the minimum and maximum expectation over the credal set $\mathcal{M}$:
$$\underline{E}(X)=\min_{p\in\mathcal{M}}E_p(X)\qquad\overline{E}(X)=\max_{p\in\mathcal{M}}E_p(X)$$
The next functions implement this.
We'll make use of the ``expectation`` function introduced previously.
In the code, we abstract the minimum and maximum transformations,
since we will consider an additional transformation later in the exercises,
and it makes for more compact and reusable code.

In [2]:
from collections.abc import Callable, Sequence

PMF = Sequence[float]
Gamble = Sequence[float]

def expectation(pmf: PMF, gamble: Gamble) -> float:
    return sum(p * g for p, g in zip(pmf, gamble))

def transform_expectations(
    transform: Callable[[Sequence[float]], float],  # sequence of expectations -> float
    credal_set: Sequence[PMF],
    gamble: Gamble,
) -> float:
    return transform([expectation(pmf, gamble) for pmf in credal_set])

def lower_expectation(credal_set: Sequence[PMF], gamble: Gamble) -> float:
    return transform_expectations(min, credal_set, gamble)

def upper_expectation(credal_set: Sequence[PMF], gamble: Gamble) -> float:
    return transform_expectations(max, credal_set, gamble)

Let's test this by finding the lower expectation of the gamble ``[5, 3, 1]`` with respect to the credal set ``[[0.2, 0.2, 0.6], [0.1, 0.1, 0.8]]``:

In [3]:
lower_expectation(
    credal_set=[[0.2, 0.2, 0.6], [0.1, 0.1, 0.8]],
    gamble=[5, 3, 1],
)

1.6

Let's check this value by evaluating the expectation of the gamble with respect to each member of the credal set:

In [4]:
[0.2 * 5 + 0.2 * 3 + 0.6 * 1, 0.1 * 5 + 0.1 * 3 + 0.8 * 1]

[2.2, 1.6]

The lower expectation is the lowest of these numbers, and indeed we can see that the correct value has been found.

**Exercise** Can you calculate the lower expectation of the gamble ``[1, 4, 2]`` with respect to the same credal set as the one used in the example?

In [5]:
# write your solution here

**Exercise** Can you also calculate the same gamble's upper expectation (again with the same credal set)?

In [6]:
# write your solution here

**Exercise** Verify that $\overline{E}(X)=-\underline{E}(-X)$, for the gamble ``[1, 4, 2]``.

In [7]:
# write your solution here

## Gamma-maximin

In $\Gamma$-maximin, we choose the gamble with the highest lower expectation:
$$\arg\max_{d\in D}\underline{E}(X_d)$$

In [8]:
def is_gamma_maxi_something(
    tol: float,  # absolute tolerance
    something: Callable[[Gamble], float],  # gamble -> float (e.g. lower prevision, upper prevision, ...)
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    values = list(map(something, gambles))
    max_value = max(values)
    return [value >= max_value - tol for value in values]

def is_gamma_maximin(
    tol: float,
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    def something(gamble: Gamble) -> float:
        return lower_expectation(credal_set, gamble)
    return is_gamma_maxi_something(tol, something, gambles)

Let us test this on our example, with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``:

In [9]:
is_gamma_maximin(
    tol=1e-6,
    credal_set=[[0.5, 0.5], [0.8 , 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[False, False, True]

This tells us that the third gamble (and only the third one) is optimal according to $\Gamma$-maximin.

## Gamma-maximax

The situation for $\Gamma$-maximax is very similar.
We choose the gamble with the highest upper expectation:
$$\arg\max_{d\in D}\overline{E}(X_d)$$
A small change in the above ``is_gamma_maximin`` function gives us ``is_gamma_maximax``:

In [10]:
def is_gamma_maximax(
    tol: float,
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    def something(gamble: Gamble) -> float:
        return upper_expectation(credal_set, gamble)  # we changed just this line
    return is_gamma_maxi_something(tol, something, gambles)

Let us again test this on our example, with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``:

In [11]:
is_gamma_maximax(
    tol=1e-6,
    credal_set=[[0.5, 0.5], [0.8 , 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[True, False, False]

This tells us that the first gamble (and only the first one) is optimal according to $\Gamma$-maximax.

## Hurwicz

We cover this criterion as part of the exercises only. Hurwicz allows a choice inbetween $\Gamma$-maximin and $\Gamma$-maximax.
First, we fix a pessimism index $\beta\in[0,1]$.
We then choose the gamble with the highest convex combination of lower and upper expectation, as follows:
$$\arg\max_{d\in D}\beta\underline{E}(X_d)+(1-\beta)\overline{E}(X_d)$$

**Exercise** Explain why $\beta$ is called a 'pessimism' index (and not, say, an 'optimism' index).

*Write your solution here.*

We can implement the Hurwicz criterion as follows:

In [12]:
def hurwicz_expectation(
    beta: float,
    credal_set: Sequence[PMF],
    gamble: Gamble,
) -> float:
    def hurwicz(expectations: Sequence[float]) -> float:
        return beta * min(expectations) + (1 - beta) * max(expectations)
    return transform_expectations(hurwicz, credal_set, gamble)

def is_hurwicz(
    tol: float,
    beta: float,
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    def something(gamble: Gamble) -> float:
        return hurwicz_expectation(beta, credal_set, gamble)
    return is_gamma_maxi_something(tol, something, gambles)

**Exercise** The ``hurwicz_expectation`` calculates part of the formula for the Hurwicz criterion. Which part is that?

*Write your solution here.*

**Exercise** Find the Hurwicz optimal gambles in our example with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``, for $\beta=0.5$. Confirm that the optimal choice here is ``[420, 300]`` (corresponding to neither the $\Gamma$-maximin nor the $\Gamma$-maximax solution).

In [13]:
# write your solution here

## Interval Maximality

In interval maximality, we first introduce an ordering $\sqsupset$ defined by
$$X\sqsupset Y\text{ whenever }\underline{E}(X)>\overline{E}(Y)$$
or in other words, whenever
the interval $[\underline{E}(X),\overline{E}(X)]$
completely dominates the interval $[\underline{E}(Y),\overline{E}(Y)]$.

Under interval maximality, we
choose any gamble which is undominated with respect to $\sqsupset$:
$$\{d\colon (\forall e\in D)(X_e\not\sqsupset X_d)\}$$
We can implement this as follows.
For guidance, a mathematical explanation of the variables used is as follows
(recall that $\mathcal{M}$ denotes the credal set):

* ``xs`` is $\{E_p(X)\colon p\in\mathcal{M}\}$ for some gamble $X$
* ``ys`` is $\{E_p(Y)\colon p\in\mathcal{M}\}$ for some gamble $Y$
* ``xss`` is $\{\{E_p(X_d)\colon p\in\mathcal{M}\}\colon d\in D\}$

The remaining variables should be obvious.

In [14]:
def interval_dominates(
    tol: float,  # tolerance
    xs: Sequence[float],  # first vector
    ys: Sequence[float],  # second vector
) -> bool:
    return min(xs) > max(ys) + tol

def is_maximal(
    dominates: Callable[[Sequence[float], Sequence[float]], bool],
    # compares two vectors
    xss: Sequence[Sequence[float]],
    # sequence of vectors
) -> Sequence[bool]:
    def is_not_dominated(xs: Sequence[float]) -> bool:
        return all(not dominates(ys, xs) for ys in xss)
    return [is_not_dominated(xs) for xs in xss]

def is_interval_maximal(
    tol: float,
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    def dominates(xs: Sequence[float], ys: Sequence[float]) -> bool:
        return interval_dominates(tol, xs, ys)
    xss = [[expectation(pmf, gamble) for pmf in credal_set] for gamble in gambles]
    return is_maximal(dominates, xss)

**Exercise** In the function ``interval_dominates``, explain what the expressions ``min(xs)`` and ``max(xs)`` correspond to.

*Write your solution here.*

Let us test this on our example, with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``:

In [15]:
is_interval_maximal(
    tol=1e-6,
    credal_set=[[0.5, 0.5], [0.8 , 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[True, True, True]

**Exercise** We proved in the lectures that the set of interval maximal gambles is equal to
$$\left\{d\colon \overline{E}(X_d)\ge\max_{e\in D}\underline{E}(X_e)\right\}$$
Can you implement this as a function?

In [16]:
def is_interval_maximal_2(
    tol: float,
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    xss = [[expectation(pmf, gamble) for pmf in credal_set] for gamble in gambles]
    maxmin = max(min(xs) for xs in xss)
    maxs = [max(xs) for xs in xss]
    return # ... complete this line!

# test the result
is_interval_maximal_2(
    tol=1e-6,
    credal_set=[[0.5, 0.5], [0.8 , 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

**Exercise** Prove that $\Gamma$-maximin, $\Gamma$-maximax, and Hurwicz gambles are always interval maximal.

## Robust Bayes Maximality

We say that $X$ robust Bayes dominates $Y$, and write $X\succ Y$,
whenever
$$\forall p\in\mathcal{M}\colon E_p(X)>E_p(Y)$$
or equivalently,
$$\underline{E}(X-Y)>0$$

Under robust Bayes maximality, we
choose any gamble which is undominated with respect to $\succ$:
$$\{d\colon (\forall e\in D)(X_e\not\succ X_d)\}$$
We can implement this as follows, recycling our ``is_maximal`` function from before:

In [17]:
def pointwise_dominates(
    tol: float,  # tolerance
    xs: Sequence[float],  # first vector
    ys: Sequence[float],  # second vector
) -> bool:
    return all(x > y + tol for x, y in zip(xs, ys))

def is_rbayes_maximal(
    tol: float,
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    def dominates(xs: Sequence[float], ys: Sequence[float]) -> bool:
        return pointwise_dominates(tol, xs, ys)
    xss = [[expectation(pmf, gamble) for pmf in credal_set] for gamble in gambles]
    return is_maximal(dominates, xss)

**Exercise** Compare the implementation of ``is_rbayes_maximal`` with the implementation of ``is_interval_maximal``. There is just one difference. Can you identify it?

Let us test this on our example, with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``:

In [18]:
is_rbayes_maximal(
    tol=1e-6,
    credal_set=[[0.5, 0.5], [0.8 , 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[True, True, True]

**Exercise** (Very hard) We saw in the lectures that we do not need to compare all gambles, because every non-maximal element is dominated by a maximal element. This holds not just for robust Bayes maximality, but for every transitive ordering. Can you improve the ``is_maximal`` function so it immediately eliminates non-maximal gambles from consideration, leading to a more efficient implementation?

## Robust Bayes Admissibility

Under robust Bayes admissibility, we choose all decisions
that have maximal expectation with respect to some $p\in\mathcal{M}$:
$$\bigcup_{p\in\mathcal{M}}\arg\max_{d\in D} E_p(X_d)$$
In code, we can implement this simply as follows:

In [19]:
def is_rbayes_admissible(
    tol: float,
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    def arg_max(pmf: PMF) -> Sequence[bool]:
        xs = [expectation(pmf, gamble) for gamble in gambles]
        max_xs = max(xs)
        return [x + tol >= max_xs for x in xs]
    def union(bss: Sequence[Sequence[bool]]) -> Sequence[bool]:
        return [any(bs) for bs in zip(*bss)]
    return union([arg_max(pmf) for pmf in credal_set])

Let us test this on our example, with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``:

In [20]:
is_rbayes_admissible(
    tol=1e-6,
    credal_set=[[0.5, 0.5], [0.8 , 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[True, False, True]

We note that robust Bayes admissibility is not just determined by the extreme points of the credal set. For this reason, normally, the credal set is assumed convex, and in that case the resulting criterion is called *E-admissibility*. The following example demonstrates this:

In [21]:
is_rbayes_admissible(
    tol=1e-6,
    credal_set=[[0.5, 0.5], [0.65, 0.35], [0.8 , 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[True, True, True]

**Exercise** Using the code below, or otherwise, confirm that ``[0.65, 0.35]`` belongs to the convex hull of ``[[0.5, 0.5], [0.8 , 0.2]]``.

In [22]:
def combine(
    alpha: float,
    xs: Sequence[float],
    ys: Sequence[float],
) -> Sequence[float]:
    return [(1 - alpha) * x + alpha * y for x, y in zip(xs, ys)]

alpha = 0  # you'll need to tweak this value
combine(alpha, [0.5, 0.5], [0.8, 0.2])

[0.5, 0.5]

## Additional Exercises

**Exercise** Consider again the same very simple example from the lectures:

> A company makes a product, and believes in increasing future demand.  The manager asks you, the decision expert, whether he should buy new machinery, use overtime, or do nothing. The upcoming year, demand can either increase or remain the same.
>
> If we buy new machinery, then the profit at the end of the year
will be $440$ (in thousands of pounds) if demand increases, and
$260$ otherwise. Alternatively, if we use overtime, then the
profit will be $420$ if demand increases, and $300$ otherwise. If
we do nothing, profit will be $370$.

We have done additional market research, and we now know that demand will increase with probability at least $0.6$, and at most $0.65$. What advice can we give the manager now? Investigate with each optimality criterion. Hint: The credal set $\mathcal{M}$ is now ``[[0.6, 0.4], [0.65, 0.35]]``.

In [23]:
# write your solution here

**Exercise** Consider the credal set ``[[1, 0], [0, 1]]`` and the gambles ``[[10, 0], [4, 4], [0, 10]]``. Show that this decision problem has a $\Gamma$-maximin gamble that is not robust Bayes admissible.

In [24]:
# write your solution here

**Exercise** Show that every $\Gamma$-maximax gamble is robust Bayes admissible.

**Exercise** Show that every robust Bayes admissible gamble is also robust Bayes maximal.